# Ipyvolume Backend Examples

## Basic Visualization

Initialize a basic visualization session and plot a brain mesh. For this purpose we need to provide subject information, e.g. `subject_id`, `subjects_dir`; which type of freesurfer surface mesh we would like to plot, e.g. 'pialed' or 'inflated'; and which hemisphere to plot or whether to plot both.

In [ ]:
import os.path as path

import mne
from mne.viz import set_3d_backend
set_3d_backend("mayavi")

from mne.viz import Brain

In [ ]:
data_path = mne.datasets.sample.data_path()

subject_id = 'sample'
subjects_dir = path.join(data_path, 'subjects')

# show both hemispheres on one plot
hemi = 'both'
surf = 'inflated'

# Call the Brain object constructor with
# parameters to initialize the visualization session.
brain = Brain(subject_id, hemi, surf, size=300, subjects_dir=subjects_dir)
brain.show()

## Activation Data Visualization

In this example it is shown how to add activation data to a plot. By default, color bar and input boxes for `fmin`, `fmid` and `fmax` control parameters of a color map are attached to the plot.

A colormap may be sequential or divergent, the latter has defined `center` parameter. The meanings of fmin, fmid and fmax are different for sequential and divergent colormaps. A sequential colormap is characterised by 
```
[fmin, fmid, fmax]
```
where `fmin` and `fmax` define the edges of the colormap and `fmid` will be the value mapped to the center of the originally chosen colormap. A divergent colormap is characterised by
```
[center-fmax, center-fmid, center-fmin, center, center+fmin, center+fmid, center+fmax]
```
i.e., values between center-fmin and center+fmin will not be shown while `center-fmid` will map to the middle of the first half of the original colormap and `center-fmid` to the middle of the second half.

In [ ]:
act_data = path.join(data_path, 'MEG/sample/sample_audvis-meg-eeg')

# read mne provided example of data
stc = mne.read_source_estimate(act_data)

# select which hemisphere to plot
hemi = 'lh'
# select data that corresponds to the 'lh' hemisphere for the selected time moment
hemi_data = stc.data[:len(stc.vertices[0]), 10]
hemi_vertices = stc.vertices[0]

fmin = stc.data.min()
fmax = stc.data.max()

In [ ]:
brain_data = Brain(subject_id, hemi, surf, size=300, subjects_dir=subjects_dir)

# color bar controls are attached by default, so we need to change parameter value
brain_data.add_data(hemi_data, fmin=fmin, hemi=hemi, fmax=fmax, colormap='hot', vertices=hemi_vertices, colorbar=False)
brain_data.show()

## Show source space
The code was taken from the [plot forward tutorial](https://www.martinos.org/mne/stable/auto_tutorials/plot_forward.html?highlight=plot_forward)

In [ ]:
import ipyvolume as ipv
import numpy as np
from mne.datasets import sample


subject = 'sample'
data_path = sample.data_path()
subjects_dir = data_path + '/subjects'

brain = Brain('sample', 'lh', 'inflated', subjects_dir=subjects_dir)
surf = brain.geo['lh']

src = mne.setup_source_space(subject, spacing='oct6',
                             subjects_dir=subjects_dir, add_dist=False)

vertidx = np.where(src[0]['inuse'])[0]

ipv.scatter(x=surf.x[vertidx], y=surf.y[vertidx], z=surf.z[vertidx], color=(1, 1, 0),
           marker='sphere', size=0.5)
brain.show()

## Display ROI Values

This example code was taken from the [PySurfer example](https://pysurfer.github.io/auto_examples/plot_parc_values.html#sphx-glr-auto-examples-plot-parc-values-py)

In [ ]:
import os
import numpy as np
import nibabel as nib


subject_id = "fsaverage"
hemi = "lh"
surf = "inflated"

"""
Bring up the visualization.
"""
brain = Brain(subject_id, hemi, surf, background=(0, 0, 0), subjects_dir=subjects_dir)

"""
Read in the automatic parcellation of sulci and gyri.
"""
aparc_file = os.path.join(subjects_dir,
                          subject_id, "label",
                          hemi + ".aparc.a2009s.annot")
labels, ctab, names = nib.freesurfer.read_annot(aparc_file)

"""
Make a random vector of scalar data corresponding to a value for each region in
the parcellation.

"""
rs = np.random.RandomState(4)
roi_data = rs.uniform(.5, .8, size=len(names))

"""
Make a vector containing the data point at each vertex.
"""
vtx_data = roi_data[labels]

"""
Handle vertices that are not defined in the annotation.
"""
vtx_data[labels == -1] = -1

"""
Display these values on the brain. Use a sequential colormap (assuming
these data move from low to high values), and add an alpha channel so the
underlying anatomy is visible.
"""
brain.add_data(vtx_data, .5, .75, colormap="jet", alpha=.8)
brain.show()